In [ ]:
#r "nuget: AWSSDK.Bedrock, 3.7.0"
#r "nuget: AWSSDK.BedrockRuntime, 3.7.0"
using System;
using System.Linq;
using System.Text.Json;
using System.Threading.Tasks;
using Amazon.Bedrock;
using Amazon.Bedrock.Model;
using Amazon.BedrockRuntime;
using Amazon.BedrockRuntime.Model;
using Amazon.Runtime;

// =====================================================================
// AWS Bedrock Authentication Demo - Minimal Version
// Supports: Access Key, Access Token, Session Token authentication
// =====================================================================

Console.WriteLine("=== AWS Bedrock Authentication Demo ===\n");

string authMethod = Environment.GetEnvironmentVariable("BEDROCK_AUTH_METHOD") ?? "session";
Console.WriteLine($"Authentication method: {authMethod}\n");

string moviePrompt = "Tell me about the movie Inception and its impact on cinema.";

AmazonBedrockClient bedrockClient = null;
AmazonBedrockRuntimeClient runtimeClient = null;

try
{
    Console.WriteLine();
    // Select authentication method
    bedrockClient = authMethod.ToLower() switch
    {
        _ => new AmazonBedrockClient(GetCredentials())
    };

    // Invoke Claude model
    Console.WriteLine("\n--- Invoking Claude Model ---\n");
    
    var invokeRequest = new InvokeModelRequest
    {
        ModelId = "anthropic.claude-3-sonnet-20240229-v1:0",
        Body = new System.IO.MemoryStream(System.Text.Encoding.UTF8.GetBytes(
            JsonSerializer.Serialize(new
            {
                anthropic_version = "bedrock-2023-05-31",
                max_tokens = 1024,
                messages = new[] 
                {
                    new 
                    { 
                        role = "user", 
                        content = moviePrompt 
                    }
                }
            })
        )),
        ContentType = "application/json",
        Accept = "application/json"
    };
    
    runtimeClient = new AmazonBedrockRuntimeClient(GetCredentials());
    var invokeResponse = await runtimeClient.InvokeModelAsync(invokeRequest);
    
    using var reader = new System.IO.StreamReader(invokeResponse.Body);
    var responseText = await reader.ReadToEndAsync();

    FormatAndPrintResponse(responseText);
}
catch (Exception ex)
{
    Console.WriteLine($"Error: {ex.Message}");
    Console.WriteLine($"Stack Trace: {ex.StackTrace}");
}
finally
{
    bedrockClient?.Dispose();
    runtimeClient?.Dispose();
}

void FormatAndPrintResponse(string responseText)
{
    var jsonOptions = new JsonSerializerOptions 
    { 
        WriteIndented = true,
        PropertyNameCaseInsensitive = true
    };

    var jsonDoc = JsonDocument.Parse(responseText);
    var formattedJson = JsonSerializer.Serialize(jsonDoc, jsonOptions);
    
    Console.WriteLine("--- Claude Response (Formatted JSON) ---\n");
    Console.WriteLine(formattedJson);
    
    // Extract and display just the text content
    if (jsonDoc.RootElement.TryGetProperty("content", out var contentArray))
    {
        Console.WriteLine("\n--- Extracted Text Content ---\n");
        foreach (var item in contentArray.EnumerateArray())
        {
            if (item.TryGetProperty("text", out var textElement))
            {
                Console.WriteLine(textElement.GetString());
            }
        }
    }
}

SessionAWSCredentials GetCredentials()
{
    string accessKey = "XXXXXXXXXXXXXX";
    string secretKey = "XXXXXXXXXXXXXX";
    string sessionToken = "XXXXXXXXXXXXXX";
    
    var credentials = new SessionAWSCredentials(accessKey, secretKey, sessionToken);
    return credentials;
}


Installed Packages AWSSDK.Bedrock, 3.7.200 AWSSDK.BedrockRuntime, 3.7.200

Inception was a 2010 science fiction film written and directed by Christopher Nolan. It starred Leonardo DiCaprio, Ellen Page, Tom Hardy, and others.

The movie was very innovative and had a huge impact on cinema for several reasons:

1. Complex Narrative Structure - The film employed a highly complex non-linear narrative structure with multiple nested dream levels. This challenged audiences and pioneered new ways of storytelling on film.

2. Visual Effects - The special effects depicting the bending of cities and gravity-defying fight scenes were groundbreaking and pushed the boundaries of what visual effects could achieve. Many new techniques had to be developed.

3. Original Concept - The premise of corporate espionage taking place within the architecture of the mind was a fresh, original concept that allowed for creativity in visuals and plot.

4. Box Office Success - Despite its cerebral and complex nature, Inception was a huge box office hit, making over $825 million worldwide. T